In [ ]:
# Import relevant librairies and python files
import os
from math import isnan
from operator import itemgetter
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm
from datetime import timedelta
os.chdir('../utils')
import utils_correlations
import utils_correlation_activations
import utils_evaluation
import utils_xgboost

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [ ]:
# Load data 
data_dir = "../data/raw_in/"
file_name = "Risques 2/data_set_challenge.csv"
mapping_name = "Risques 2/final_mapping_candidat.csv"

df = pd.read_csv(os.path.join(data_dir, file_name), index_col=0)
mapping = pd.read_csv(os.path.join(data_dir, mapping_name))

# --- step 1: identify the different types of series
df.columns = [str(typ) + "_" + str(col) for col, typ in zip(df.columns, mapping.Type)]

We use the functions from utils_correlation to compute the correlation coefficients between the series

In [ ]:
growth_rates_df = utils_correlations.get_growth_rates_df(df)
corr_df = utils_correlations.get_correlations_df(df)
corr_df_activated = utils_correlations.activate_correlations(corr_df,lambda x: utils_correlation_activations.truncate_below_threshold(x))
corr_df_activated.head(10)

For each time series, we will use the other time series from the dataframe to build the features for the model. for each model, we only use the time series with a correlation coefficient higher than 0.7 with the original one.

In [ ]:
# Build a dictionnary holding for each TS the other TS that will be used for training the model
feature_map = {}
for i, row in corr_df_activated.iterrows():
    features = []
    for col in row.index:
        if (row[col] < 1 and abs(row[col])>0.7):
            features.append(col)
    feature_map[i] = features

# Histogram of nb of feature per model

nb_feature = [len(x) for x in feature_map.values()]
n, bins, patches = plt.hist(x=nb_feature, bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('number of features for regression')
plt.ylabel('Frequency')
plt.title('number of features for regression')
plt.text(23, 45, r'$\mu=15, b=3$')
maxfreq = n.max()
# Set a clean upper y-axis limit.
plt.ylim(ymax=np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10)


Number of series without a feature: (because no TS is correlated enough)

In [ ]:
tot = 0

for x in nb_feature:
    if x == 0:
        tot = tot + 1
tot


Now for each series we will fill in the missing values by using the predicytions of a model that uses only the relevant features from the feature map above


In order to evaluate our model we first need to get a valid evualuation set. We then fill in the missing values by building a model for every TS except the 302 ones without features.

In [ ]:
df_full,df_miss = utils_preprocessing.get_evaluation_set(df.reset_index().drop("Date", axis=1), method="linear")
antoine_df = df_full.copy()

k=0

for x in feature_map.keys():
    k = k+1
    antoine_df[x] = utils_xgboost.model_for_series(x, feature_map[x], df_miss)


We impute the values for the 302 TS left by linear interpolation (ie same method as benchmark)

In [ ]:
mask = antoine_df.isnull()

for val, idx in antoine_df.apply(pd.Series.first_valid_index).items():
    mask.loc[idx, val] = True

for val, idx in antoine_df.apply(pd.Series.last_valid_index).items():
    mask.loc[idx, val] = True

ix = [
    (row, col)
    for row, col in zip(
        np.where(mask == False)[0],
        np.where(mask == False)[1])
]


# --- impute the missing data
antoine_dff = antoine_df.interpolate(
    method="linear", limit=None, limit_direction="forward"
)

We now create a benchmark prediction by imputing the missing values on the original evaluation set by linear interpolation

In [ ]:
df_miss_bench = df_miss.copy()

mask = df_miss_bench.isnull()
for val, idx in df_miss_bench.apply(pd.Series.first_valid_index).items():
    mask.loc[idx, val] = True

for val, idx in df_miss_bench.apply(pd.Series.last_valid_index).items():
    mask.loc[idx, val] = True

ix = [
    (row, col)
    for row, col in zip(
        np.where(mask == False)[0],
        np.where(mask == False)[1])
]


# --- impute the missing data
df_miss_bench_res = df_miss_bench.interpolate(
    method="linear", limit=None, limit_direction="forward"
)


In [ ]:
# We use the evaluation methods to get the NRMSE.

bench_dict, b, c = utils_evaluation.eval_imputation(
    df_full, df_miss_bench_res, df_miss)
print(b)


xgb_dict, e, f = utils_evaluation.eval_imputation(
    df_full, antoine_dff, df_miss)
print(e)

we graph the NRMSE improvement of the xgboost method for the series with an improvement of at least 35% compared to benchmark

In [ ]:
compar = {}
for key in bench_dict.keys():
    compar[key] = ((bench_dict[key][0] - xgb_dict[key][0])/bench_dict[key][0])*100.0
x = 0
better_xgb = []
for k in compar.keys():
    if compar[k] > 35.0:
        better_xgb.append((k, compar[k]))

better_xgb.sort(key=itemgetter(1))

y = [t[1] for t in better_xgb]
xx = [t[0] for t in better_xgb]
x = [t.split('_')[0] + ' ' + t.split('_')[len(t.split('_'))-1] for t in xx]

fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
MEDIUM_SIZE = 14
SMALL_SIZE = 10


plt.xticks(rotation=90)
plt.title('NRMSE improvment over Benchmark')
plt.xlabel('Series')
plt.ylabel('Improvement over Benchmark (%)')
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=16)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=10)    # fontsize of the tick labels
plt.rc('ytick', labelsize=14)
ax.bar(x, y)
plt.show()